# COVID-19 U.S. State-level and City-level Trends

In [1]:
%matplotlib inline

import os
import glob
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker

# Plot configurations
sns.set_context("notebook", font_scale=1.2, rc={"lines.linewidth": 1.2})
sns.set_style("darkgrid")
custom_style = {
            'grid.color': '0.8',
            'grid.linestyle': '--',
            'grid.linewidth': 0.5,
}
sns.set_style(custom_style)

### Run Parameters

In [2]:
# Path to input csvs
DATA_DIR = "csse_covid_19_daily_reports/"

# Date range to cover
MIN_DATE = "2020-03-18"
MAX_DATE = "2020-04-03"

# Add these back after rank filtering
STATE_INCLUDE_LIST = []
CITY_INCLUDE_LIST = []

# Exclude globally
STATE_EXCLUDE_LIST = []
CITY_EXCLUDE_LIST = []

# Manually set states or cities globally
STATE_OVERRIDE_LIST = []
CITY_OVERRIDE_LIST = []

# Limit to top n on max input date
CITY_LIMIT = 0
STATE_LIMIT = 5

### Sample Input Data

In [3]:
# Data sample pre-3/22

pd.read_csv(DATA_DIR + "03-21-2020.csv").head()

,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered,Latitude,Longitude
0,Hubei,China,2020-03-21T10:13:08,67800,3139,58946,30.9756,112.2707
1,NaN,Italy,2020-03-21T17:43:03,53578,4825,6072,41.8719,12.5674
2,NaN,Spain,2020-03-21T13:13:30,25374,1375,2125,40.4637,-3.7492
3,NaN,Germany,2020-03-21T20:43:02,22213,84,233,51.1657,10.4515
4,NaN,Iran,2020-03-21T11:13:12,20610,1556,7635,32.4279,53.6880


In [4]:
# Data sample post-3/22

pd.read_csv(DATA_DIR + "03-22-2020.csv").head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key
0,36061.0,New York City,New York,US,3/22/20 23:45,40.767273,-73.971526,9654,63,0,0,"New York City, New York, US"
1,36059.0,Nassau,New York,US,3/22/20 23:45,40.740665,-73.589419,1900,4,0,0,"Nassau, New York, US"
2,36119.0,Westchester,New York,US,3/22/20 23:45,41.162784,-73.757417,1873,0,0,0,"Westchester, New York, US"
3,36103.0,Suffolk,New York,US,3/22/20 23:45,40.883201,-72.801217,1034,9,0,0,"Suffolk, New York, US"
4,36087.0,Rockland,New York,US,3/22/20 23:45,41.150279,-74.025605,455,1,0,0,"Rockland, New York, US"


### Data Loading and Transormation

In [5]:
def build_aggregate_df():

    agg_df = None

    for fn in glob.glob(DATA_DIR + "*.csv"):
        df = pd.read_csv(fn)
        df["Date"] = fn.split("/")[-1].split(".csv")[0]
        df.rename(columns={
            "Country/Region": "Country_Region", 
            "Province/State": "Province_State",
            "Lat": "Latitude",
            "Long_": "Longitude",
            "Last Update": "Last_Update",
            "Admin2": "City"
        }, inplace=True)
        if agg_df is None:
            agg_df = df
        else:
            agg_df = agg_df.append(df, ignore_index=True)

    agg_df["Date"] = pd.to_datetime(agg_df["Date"])
    
    # Limit date range
    agg_df = agg_df[agg_df["Date"].between(MIN_DATE, MAX_DATE)]
    
    # Filter for U.S. only
    agg_df = agg_df[agg_df["Country_Region"]=="US"]
    
    # Filter exclusion lists
    agg_df = agg_df[~agg_df.Province_State.isin(STATE_EXCLUDE_LIST)]
    agg_df = agg_df[~agg_df.City.isin(CITY_EXCLUDE_LIST)]
    
    if STATE_OVERRIDE_LIST:
        agg_df = agg_df[agg_df.Province_State.isin(STATE_OVERRIDE_LIST)]
        
    if CITY_OVERRIDE_LIST:
        agg_df = agg_df[agg_df.City.isin(CITY_OVERRIDE_LIST)]

    print("Num Rows:", agg_df.shape[0])
    print("Total Dates:", agg_df["Date"].unique().shape[0])
    print("Min Date:", agg_df["Date"].min())
    print("Max Date:", agg_df["Date"].max())
    
    return agg_df

agg_df = build_aggregate_df()

agg_df.head()

Num Rows: 37932
Total Dates: 17
Min Date: 2020-03-18 00:00:00
Max Date: 2020-04-03 00:00:00


,FIPS,City,Province_State,Country_Region,Last_Update,Latitude,Longitude,Confirmed,Deaths,Recovered,Active,Combined_Key,Date
0,45001.0,Abbeville,South Carolina,US,2020-03-23 23:19:34,34.223334,-82.461707,1.0,0.0,0.0,0.0,"Abbeville, South Carolina, US",2020-03-23
1,22001.0,Acadia,Louisiana,US,2020-03-23 23:19:34,30.295065,-92.414197,1.0,0.0,0.0,0.0,"Acadia, Louisiana, US",2020-03-23
2,51001.0,Accomack,Virginia,US,2020-03-23 23:19:34,37.767072,-75.632346,1.0,0.0,0.0,0.0,"Accomack, Virginia, US",2020-03-23
3,16001.0,Ada,Idaho,US,2020-03-23 23:19:34,43.452658,-116.241552,13.0,0.0,0.0,0.0,"Ada, Idaho, US",2020-03-23
4,19001.0,Adair,Iowa,US,2020-03-23 23:19:34,41.330756,-94.471059,1.0,0.0,0.0,0.0,"Adair, Iowa, US",2020-03-23


In [6]:
def build_state_df(agg_df):
    
    # Aggregate at date, state grain
    state_df = agg_df[["Date", "Province_State", "Confirmed"]]
    state_df = state_df.groupby(by=["Date", "Province_State"]).sum()
    state_df = state_df.reset_index()

    # Limit to top n states
    if STATE_LIMIT <= 0:
        return state_df
    
    include = (
        state_df[state_df.Date==MAX_DATE]
        .groupby('Province_State')
        .sum()
        .sort_values("Confirmed", ascending=False)
    )[:STATE_LIMIT].index.to_list()
    
    include.extend(STATE_INCLUDE_LIST)
    
    print("Included states:\n", include)
    state_df = state_df[state_df['Province_State'].isin(include)]
    
    return state_df

state_df = build_state_df(agg_df)

state_df.head()

Included states:
 ['New York', 'New Jersey', 'Michigan', 'California', 'Massachusetts']


,Date,Province_State,Confirmed
4,2020-03-18,California,751.0
24,2020-03-18,Massachusetts,218.0
25,2020-03-18,Michigan,83.0
33,2020-03-18,New Jersey,267.0
35,2020-03-18,New York,2495.0


In [7]:
def build_city_df(agg_df):
    
    # Aggregate at date, city grain
    city_df = agg_df[["Date", "City", "Confirmed"]]
    city_df = city_df.groupby(by=["Date", "City"]).sum()
    city_df = city_df.reset_index()
    
    # Limit to top n cities
    if CITY_LIMIT < 1:
        return city_df
    
    include = (
        city_df[city_df.Date==MAX_DATE]
        .groupby('City')
        .sum()
        .sort_values("Confirmed", ascending=False)
    )[:CITY_LIMIT].index.to_list()
    
    include.extend(CITY_INCLUDE_LIST)
    
    print("Included cities:\n", include)
    city_df = city_df[city_df['City'].isin(include)]
    
    return city_df

city_df = build_city_df(agg_df)

city_df.head()

,Date,City,Confirmed
0,2020-03-22,Abbeville,1.0
1,2020-03-22,Acadia,0.0
2,2020-03-22,Accomack,1.0
3,2020-03-22,Ada,11.0
4,2020-03-22,Adair,2.0


In [8]:
def build_cumulative_df(locale_df, locale_column="Province_State"):
    cum_df = locale_df.groupby(by=["Date", locale_column]).sum()
    cum_df = cum_df.unstack()
    cum_df.columns = cum_df.columns.droplevel()
    cum_df.reset_index(level=0, inplace=True)
    
    return cum_df
    
cum_state_df = build_cumulative_df(state_df, "Province_State")

cum_state_df.head()

Province_State,Date,California,Massachusetts,Michigan,New Jersey,New York
0,2020-03-18,751.0,218.0,83.0,267.0,2495.0
1,2020-03-19,952.0,328.0,334.0,742.0,5365.0
2,2020-03-20,1177.0,413.0,552.0,890.0,8310.0
3,2020-03-21,1364.0,525.0,788.0,1327.0,11710.0
4,2020-03-22,1646.0,646.0,1035.0,1914.0,15800.0


In [9]:
cum_city_df = build_cumulative_df(city_df, "City")

cum_city_df.head()

City,Date,Abbeville,Acadia,Accomack,Ada,Adair,Adams,Addison,Aiken,Aitkin,...,Yolo,York,Young,Yuba,Yukon-Koyukuk,Yuma,Zapata,Zavala,Ziebach,unassigned
0,2020-03-22,1.0,0.0,1.0,11.0,2.0,27.0,3.0,1.0,0.0,...,6.0,27.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,NaN
1,2020-03-23,1.0,1.0,1.0,13.0,3.0,30.0,6.0,1.0,0.0,...,6.0,34.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,NaN
2,2020-03-24,1.0,2.0,1.0,19.0,2.0,37.0,8.0,2.0,0.0,...,8.0,51.0,0.0,3.0,0.0,3.0,0.0,0.0,0.0,NaN
3,2020-03-25,3.0,2.0,2.0,24.0,4.0,39.0,9.0,2.0,0.0,...,10.0,69.0,1.0,0.0,0.0,4.0,0.0,0.0,0.0,NaN
4,2020-03-26,3.0,3.0,2.0,39.0,4.0,66.0,12.0,2.0,0.0,...,11.0,75.0,1.0,3.0,0.0,6.0,0.0,0.0,0.0,NaN


In [10]:
def build_day_over_day_df(df, locale_column="Province_State"):
    df.sort_values("Date", inplace=True)
    df["_shifted"] = df.groupby(locale_column)["Confirmed"].transform(lambda x: x.shift())
    df["D/D Growth (Cases)"] = df["Confirmed"] - df["_shifted"]
    df["D/D Growth (%)"] = 100. * (df["D/D Growth (Cases)"] / df["_shifted"])
    df.drop(["_shifted"], axis=1, inplace=True)

    return df

day_over_day_state_df = build_day_over_day_df(state_df, "Province_State")

day_over_day_state_df.sort_values(["Province_State", "Date"]).head()

,Date,Province_State,Confirmed,D/D Growth (Cases),D/D Growth (%)
4,2020-03-18,California,751.0,NaN,NaN
62,2020-03-19,California,952.0,201.0,26.764314
120,2020-03-20,California,1177.0,225.0,23.634454
177,2020-03-21,California,1364.0,187.0,15.887850
235,2020-03-22,California,1646.0,282.0,20.674487


In [11]:
day_over_day_city_df = build_day_over_day_df(city_df, "City")

day_over_day_city_df.head()

,Date,City,Confirmed,D/D Growth (Cases),D/D Growth (%)
0,2020-03-22,Abbeville,1.0,NaN,NaN
1241,2020-03-22,Otero,0.0,NaN,NaN
1240,2020-03-22,Oswego,0.0,NaN,NaN
1239,2020-03-22,Oscoda,0.0,NaN,NaN
1238,2020-03-22,Osceola,24.0,NaN,NaN


In [12]:
def build_normalized_state_df(cum_df, start_n=200):
    normalized_df = pd.DataFrame(columns=["Province_State", "Confirmed"])

    for state in cum_df.columns[1:]:
        df = cum_df[state].to_frame()
        df.columns = ["Confirmed"]
        df = df[df.Confirmed>start_n].reset_index(drop=True)
        df["Province_State"] = state
        normalized_df = normalized_df.append(df)
        
    normalized_df['Date'] = normalized_df.index
    normalized_df = build_cumulative_df(normalized_df, "Province_State")
        
    return normalized_df
        
normalized_state_df = build_normalized_state_df(cum_state_df)
    
normalized_state_df.head()

Province_State,Date,California,Massachusetts,Michigan,New Jersey,New York
0,0,751.0,218.0,334.0,267.0,2495.0
1,1,952.0,328.0,552.0,742.0,5365.0
2,2,1177.0,413.0,788.0,890.0,8310.0
3,3,1364.0,525.0,1035.0,1327.0,11710.0
4,4,1646.0,646.0,1329.0,1914.0,15800.0


In [ ]:
def build_normalized_city_df(cum_df, start_n=100):
    normalized_df = pd.DataFrame(columns=["City", "Confirmed"])

    for city in cum_df.columns[1:]:
        df = cum_df[city].to_frame()
        df.columns = ["Confirmed"]
        df = df[df.Confirmed>start_n].reset_index(drop=True)
        df["City"] = city
        normalized_df = normalized_df.append(df)
        
    normalized_df['Date'] = normalized_df.index
    normalized_df = build_cumulative_df(normalized_df, "City")
        
    return normalized_df
        
normalized_city_df = build_normalized_city_df(cum_city_df)
    
normalized_city_df.head()

In [ ]:
def legend_positions(df, y):
    """ Calculate position of labels to the right in plot... """
    positions = {}
    
    for column in y:
        positions[column] = df[df[column].notnull()][column].values[-1] - 0.5  

    def push():
        """
        ...by putting them to the last y value and
        pushing until no overlap
        """
        collisions = 0
        for column1, value1 in positions.items():
            for column2, value2 in positions.items():
                if column1 != column2:
                    dist = abs(value1-value2)
                    if dist < 2.5:
                        collisions += 1
                        if value1 < value2:
                            positions[column1] -= .1
                            positions[column2] += .1
                        else:
                            positions[column1] += .1
                            positions[column2] -= .1
                        return True
    while True:
        pushed = push()
        if not pushed:
            break

    return positions

In [ ]:
def plot_case_growth(df):
    plot_df = df.copy()
    x = 'Date'
    y = plot_df.columns.tolist()[1:]
    
    f, ax = plt.subplots(figsize=(14, 14))        
    cmap = plt.cm.get_cmap('Dark2', len(y))
    
    # Generate positions
    positions = legend_positions(plot_df, y)
    
    for i, (column, position) in enumerate(positions.items()):

        # Get a color
        color = cmap(float(i)/len(positions))
        # Plot each line separatly so we can be explicit about color
        ax = plot_df.plot(x=x, y=column, legend=False, ax=ax, color=color)

        # Add the text to the right
        plt.text(
            plot_df[x][plot_df[column].last_valid_index()],
            position, column, fontsize=12,
            color=color # Same color as line
        )
    ax.set_ylabel('Cumulative Confirmed Cases')
    
    if plot_df.Date.dtype == "int":
        ax.set(xlim=(0, plot_df.Date.max() + 1))
        ax.set_xlabel('Num Days')
    else:
        ax.set(xlim=(MIN_DATE, plot_df.Date.max() + pd.DateOffset(days=1)))

    sns.despine()

## Data Visualization

### Cumulative Cases By State

In [ ]:
plot_case_growth(cum_state_df)

### Cumulative Cases By State (Normalized Start Date)

In [ ]:
plot_case_growth(build_normalized_state_df(cum_state_df, start_n=500))

### Cumulative Cases By City

In [ ]:
if len(cum_city_df.columns) < 49:
    plot_case_growth(cum_city_df)
else:
    print("Too Many Cities to Display")

### Cumulative Cases By City (Normalized Start Date)

In [ ]:
if len(cum_city_df.columns) < 49:
    plot_case_growth(build_normalized_city_df(cum_city_df, start_n=500))
else:
    print("Too Many Cities to Display")

### Daily Growth Rates

In [ ]:
def plot_daily_growth(df, locale, locale_col="Province_State", tick_interval=None):
    plot_df = df[df[locale_col]==locale].copy()
    
    if plot_df.empty:
        return "No Data Found For Given Inputs and Filters"
    
    print(plot_df)
    
    plot_df["Date"] = plot_df["Date"].apply(lambda x: x.strftime("%m-%d"))

    fig, ax1 = plt.subplots(figsize=(14, 12))
    ax2 = ax1.twinx()

    color = 'tab:blue'
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Cases', color=color)
    
    # Puts ticks at regular intervals
    if tick_interval:
        loc = plticker.MultipleLocator(base=tick_interval)
        ax1.yaxis.set_major_locator(loc)
        
    ax1.bar(plot_df.Date, plot_df["Confirmed"], color=color)

    color = 'tab:green'
    ax1.bar(plot_df.Date, plot_df["D/D Growth (Cases)"], color=color)
    ax1.legend(["Total Cases", "New Cases"])

    color = 'tab:red'
    ax2.set_ylabel('% Of Total Cases', color=color)
    ax2.set_ylim(0, 100)
    ax2.plot(plot_df.Date, plot_df["D/D Growth (%)"], color=color, linewidth=4)
    ax2.grid(False)

    plt.title(f"{locale} Case Growth")

    plt.show()

In [ ]:
plot_daily_growth(day_over_day_state_df, locale="New York", locale_col="Province_State", tick_interval=10000.0)

In [ ]:
plot_daily_growth(day_over_day_city_df, locale="New York City", locale_col="City", tick_interval=5000.0)

In [ ]:
plot_daily_growth(day_over_day_state_df, locale="California", locale_col="Province_State", tick_interval=1000.0)

In [ ]:
plot_daily_growth(day_over_day_city_df, locale="San Francisco", locale_col="City", tick_interval=500.0)